1.
Mean–Variance (MV) optimization requires knowing each asset’s expected return and how every pair of assets moves together, that is, their covariances.
With 1,000 securities, you’d need to estimate:
1,000 expected returns, and

roughly ½ × 1,000 × (1,000 – 1) ≈ 500,000 covariances.

2.
For the two-stage optimization to approximate the full mean–variance optimization well, the division of securities into asset classes must group together assets that are highly correlated with each other but weakly correlated with assets in other classes.
In other words, the asset-class partition must reflect genuinely distinct sources of risk and return so that each class behaves almost like a single factor.

3.
TIPS should be treated as a separate asset class rather than grouped with nominal bonds.
Although they are government bonds, their risk and return characteristics differ fundamentally:

4.
HMC focuses on real returns because the university’s spending and obligations rise with inflation. What matters is how much the endowment’s purchasing power grows — not just the nominal dollar value.
If inflation is 8% and the portfolio earns 10%, the real gain is only about 2%.
 So real returns show the true economic performance of the portfolio.

In [44]:
import pandas as pd
import numpy as np
import os

In [45]:
file = "/Users/jacopomichelacci/AlgoPython/data/class_data/multi_asset_etf_data.xlsx"

#loading excess ret sheet
df_exc_ret = pd.read_excel(file, sheet_name="excess returns")

#drop QAI, SHV and Date
df_exc_ret = df_exc_ret.drop(columns=["QAI", "SHV", "Date"], errors="ignore")

In [46]:
#2) 1. Summary Statistics
mean_monthly_r = df_exc_ret.mean()
vol_monthly_r = df_exc_ret.std()

mean_annual_r = mean_monthly_r * 12
vol_annual_r = vol_monthly_r * (12 ** 0.5)
sharpe = mean_annual_r / vol_annual_r

summary = pd.DataFrame({
    "mean (annual)" : mean_annual_r,
    "vol (annual)" : vol_annual_r,
    "sharpe" : sharpe
}).sort_values("sharpe", ascending=False)

print(summary)

     mean (annual)  vol (annual)    sharpe
SPY       0.128141      0.142839  0.897103
HYG       0.041371      0.075928  0.544873
IYR       0.074916      0.168675  0.444143
PSP       0.092561      0.213370  0.433804
EFA       0.061775      0.150903  0.409372
TIP       0.020502      0.051115  0.401091
IEF       0.016404      0.063442  0.258569
EEM       0.029339      0.176164  0.166542
DBC      -0.005292      0.166553 -0.031774
BWX      -0.007716      0.082789 -0.093202


In [47]:
#2) 2. Descriptive Analysis

corr_matrix = df_exc_ret.corr()
print(corr_matrix)

          BWX       DBC       EEM       EFA       HYG       IEF       IYR  \
BWX  1.000000  0.191116  0.621673  0.602820  0.602555  0.580891  0.552557   
DBC  0.191116  1.000000  0.511667  0.500922  0.461887 -0.300207  0.280518   
EEM  0.621673  0.511667  1.000000  0.819925  0.691167  0.026704  0.584063   
EFA  0.602820  0.500922  0.819925  1.000000  0.787191  0.042639  0.699292   
HYG  0.602555  0.461887  0.691167  0.787191  1.000000  0.187258  0.739356   
IEF  0.580891 -0.300207  0.026704  0.042639  0.187258  1.000000  0.316532   
IYR  0.552557  0.280518  0.584063  0.699292  0.739356  0.316532  1.000000   
PSP  0.526692  0.453303  0.750109  0.895320  0.812157  0.022436  0.749836   
SPY  0.439994  0.432162  0.687751  0.845863  0.793518  0.000815  0.754711   
TIP  0.675151  0.109006  0.378792  0.394821  0.538648  0.754102  0.598742   

          PSP       SPY       TIP  
BWX  0.526692  0.439994  0.675151  
DBC  0.453303  0.432162  0.109006  
EEM  0.750109  0.687751  0.378792  
EFA  0.8

In [48]:
#create ones matrix and keep only upper triangle
corr_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

#unstack matrix
corr_unstacked = corr_matrix.unstack().dropna()

highest_corr_pair = corr_unstacked.idxmax()
lowest_corr_pair = corr_unstacked.idxmin()

print(f"\nHighest correlation pair: {highest_corr_pair} {corr_unstacked.max():.3f}")
print(f"Lowest correlation pair: {lowest_corr_pair} {corr_unstacked.min():.3f}\n")

#TIPS observation
print(f"TIPS Sharpe: 0.401\nIEF Sharpe: 0.258\nBWX Sharpe: -0.093")
print(f"\nTIPS Have Greatly Outperformed Domestic and Foreign Bonds")


Highest correlation pair: ('PSP', 'EFA') 0.895
Lowest correlation pair: ('IEF', 'DBC') -0.300

TIPS Sharpe: 0.401
IEF Sharpe: 0.258
BWX Sharpe: -0.093

TIPS Have Greatly Outperformed Domestic and Foreign Bonds


In [ ]:
#2) 3. The MV frontier
cov_matrix = df_exc_ret.cov()

mu = mean_monthly_r.values.reshape(-1,1)
ones = np.ones_like(mu)
inv_cov = np.linalg.inv(cov_matrix.values)

w_unnormalized = inv_cov @ mu
denominator = ones.T @ inv_cov @ mu

w_tan = w_unnormalized / denominator

df_w_tan = pd.DataFrame(
    w_tan,
    index=df_exc_ret.columns,
    columns=["Tangency weight"]
).sort_values("Tangency weight", ascending=False)

print(df_w_tan)

#mean, vol, sharpe annualized
mean_port = (mu.T @ w_tan).item() * 12
vol_port = np.sqrt((w_tan.T @ cov_matrix.values @ w_tan).item()) * (12 ** 0.5)
sharpe_port = mean_port / vol_port

print(f"\nPortfolio mean: {mean_port:.3%}")
print(f"Portfolio vol:  {vol_port:.3%}")
print(f"Portfolio Sharpe: {sharpe_port:.3f}")



[[-0.00064301]
 [-0.00044101]
 [ 0.00244489]
 [ 0.00514795]
 [ 0.00344759]
 [ 0.001367  ]
 [ 0.00624299]
 [ 0.00771338]
 [ 0.01067844]
 [ 0.00170849]]
     Tangency weight
SPY         1.059632
IEF         0.881186
HYG         0.290614
TIP         0.175293
EFA         0.068682
EEM         0.026437
DBC        -0.071623
IYR        -0.246582
PSP        -0.332995
BWX        -0.850643

Portfolio mean: 12.852%
Portfolio vol:  8.748%
Portfolio Sharpe: 1.469


In [50]:
#2) 4. TIPS
df_no_tips = df_exc_ret.drop(columns=["TIP"])

cov_no_tips = df_no_tips.cov()

mu_no_tips = mean_monthly_r[df_no_tips.columns].values.reshape(-1,1)
ones_no_tips = np.ones_like(mu_no_tips)
inv_cov_no_tips = np.linalg.inv(cov_no_tips.values)

w_unnorm_no_tips = inv_cov_no_tips @ mu_no_tips
denominator_no_tips = ones_no_tips.T @ inv_cov_no_tips @ mu_no_tips

w_tan_no_tips = w_unnorm_no_tips / denominator_no_tips

# portfolio stats
mean_port_no_tips = (mu_no_tips.T @ w_tan_no_tips).item() * 12
vol_port_no_tips  = np.sqrt((w_tan_no_tips.T @ cov_no_tips.values @ w_tan_no_tips).item()) * (12 ** 0.5)
sharpe_no_tips = mean_port_no_tips / vol_port_no_tips

print(f"Portfolio (no TIPS) mean: {mean_port_no_tips:.3%}")
print(f"Portfolio (no TIPS) vol:  {vol_port_no_tips:.3%}")
print(f"Portfolio (no TIPS) Sharpe: {sharpe_no_tips:.3f}")

#case B --- raise TIPS expected return by 0.0012 --- 
mu_adj = mean_monthly_r.copy()
mu_adj["TIP"] += 0.0012


mu_adj_vec = mu_adj.values.reshape(-1,1)
ones = np.ones_like(mu_adj_vec)

inv_cov = np.linalg.inv(cov_matrix.values)
w_unnorm_adj = inv_cov @ mu_adj_vec
denominator_adj = ones.T @ inv_cov @ mu_adj_vec
w_tan_adj = w_unnorm_adj / denominator_adj

# portfolio stats
mean_port_adj = (mu_adj_vec.T @ w_tan_adj).item() * 12
vol_port_adj  = np.sqrt((w_tan_adj.T @ cov_matrix.values @ w_tan_adj).item()) * (12 ** 0.5)
sharpe_adj = mean_port_adj / vol_port_adj

print(f"\nPortfolio (TIPS +0.0012) mean: {mean_port_adj:.3%}")
print(f"Portfolio (TIPS +0.0012) vol:  {vol_port_adj:.3%}")
print(f"Portfolio (TIPS +0.0012) Sharpe: {sharpe_adj:.3f}")

Portfolio (no TIPS) mean: 13.274%
Portfolio (no TIPS) vol:  9.045%
Portfolio (no TIPS) Sharpe: 1.467

Portfolio (TIPS +0.0012) mean: 12.036%
Portfolio (TIPS +0.0012) vol:  7.465%
Portfolio (TIPS +0.0012) Sharpe: 1.612


Removing TIPS caused almost no change in portfolio Sharpe (≈ same as the base case), suggesting that at historical returns, TIPS do not materially improve diversification.
However, when the expected excess return of TIPS was increased by 0.0012, the portfolio Sharpe ratio rose of about 0.143

In [51]:
#3) EW PORTFOLIO

#target monthly excess return
target_mean = 0.01

#Equally Weighted Portfolio
n_assets = len(df_exc_ret.columns)
w_ew = np.ones(n_assets) / n_assets

#scaling to target_mean
mean_ew = (mean_monthly_r @ w_ew)
scale_ew = target_mean / mean_ew
w_ew_scaled = w_ew * scale_ew


#Risk Parity portfolio
var_assets = df_exc_ret.var()
inv_var = 1 / var_assets
w_rp = inv_var / inv_var.sum()

#scale to target_mean
mean_rp = (mean_monthly_r @ w_rp)
scale_rp = target_mean / mean_rp
w_rp_scaled = w_rp * scale_rp


#Mean Variance Portfolio
mean_mv = (mean_monthly_r @ w_tan.flatten())
scale_mv = target_mean / mean_mv
w_mv_scaled = w_tan.flatten() * scale_mv

#comparison
portfolio = {
    "Equally Weighted" : w_ew_scaled,
    "Risk Parity" : w_rp_scaled,
    "Mean Variance" : w_mv_scaled
}

results = []

for name, w in portfolio.items():
    port_rets = df_exc_ret @ w

    mean_annual = port_rets.mean() * 12
    vol_annual = port_rets.std() * (12 ** 0.5)
    sharpe = mean_annual / vol_annual

    results.append([name, mean_annual, vol_annual, sharpe])

results_df = pd.DataFrame(
    results,
    columns=["Portfolio", "Mean Ret(annual)", "Vol Ret(annual)", "Sharpe"]
)

print(results_df)

          Portfolio  Mean Ret(annual)  Vol Ret(annual)    Sharpe
0  Equally Weighted              0.12         0.269247  0.445687
1       Risk Parity              0.12         0.263919  0.454685
2     Mean Variance              0.12         0.081678  1.469190
